In [1]:
import os
import numpy as np
import tensorflow as tf
tf.get_logger().setLevel('ERROR')  # Or 'WARNING'

import tensorflow_datasets as tfds
tfds.disable_progress_bar()

print(f'You use TensorFlow v.{tf.__version__}.')

2025-08-26 02:56:33.613669: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756173393.636010 2043406 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756173393.643500 2043406 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1756173393.660640 2043406 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756173393.660668 2043406 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756173393.660670 2043406 computation_placer.cc:177] computation placer alr

You use TensorFlow v.2.19.0.


In [2]:
# gpus = tf.config.list_physical_devices('GPU')
# if gpus:
#     try:
#         for gpu in gpus:
#             tf.config.experimental.set_memory_growth(gpu, True)
#     except RuntimeError as e:
#         print("Error setting GPU memory growth:", e)

In [3]:
# Path to store tensorflow datasets
data_dir = '/data/tensorflow_datasets'

# Load MNIST, store it in the specified folder
datasets, info = tfds.load(
    name='mnist',
    data_dir=data_dir,       
    with_info=True,
    as_supervised=True
)

mnist_train, mnist_test = datasets['train'], datasets['test']

I0000 00:00:1756173397.320699 2043406 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 878 MB memory:  -> device: 0, name: NVIDIA RTX 6000 Ada Generation, pci bus id: 0000:09:00.0, compute capability: 8.9
I0000 00:00:1756173397.323130 2043406 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 881 MB memory:  -> device: 1, name: NVIDIA RTX 6000 Ada Generation, pci bus id: 0000:42:00.0, compute capability: 8.9


In [4]:
strategy = tf.distribute.MirroredStrategy()
print(f'Number of devices the network trains on: {strategy.num_replicas_in_sync}')

Number of devices the network trains on: 2


In [5]:
# Get the number of sample per split
num_train_examples = info.splits['train'].num_examples  
num_test_examples = info.splits['test'].num_examples   
total_examples = info.splits.total_num_examples       

# Print dataset summary
print(
    f'Dataset summary:\n'
    f'  Total samples: {total_examples}.\n'
    f'  Training samples: {num_train_examples}.\n'
    f'  Test samples: {num_test_examples}.'
)


BUFFER_SIZE = 10_000  # For shuffling

# Define buffer and batch size for training
BATCH_SIZE_PER_REPLICA = 64

# Global batch size for multi-GPU. Adjust buffer size for distributed training if using MirroredStrategy
BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync

Dataset summary:
  Total samples: 70000.
  Training samples: 60000.
  Test samples: 10000.


In [6]:
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255

    return image, label



train_dataset = (
    mnist_train
    .map(scale)
    .cache()
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE)
    .prefetch(tf.data.AUTOTUNE)
)

eval_dataset = (
    mnist_test
    .map(scale)
    .batch(BATCH_SIZE)
    .prefetch(tf.data.AUTOTUNE)
)

In [7]:
with strategy.scope():
    model = tf.keras.Sequential([
        tf.keras.Input(shape=(28, 28, 1)),
        tf.keras.layers.Conv2D(32, 3, activation='relu'),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(10)
    ])

    model.compile(
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        optimizer=tf.keras.optimizers.Adam(),
        metrics=['accuracy']
    )

In [8]:
model.fit(train_dataset, epochs=12)

W0000 00:00:1756173398.276204 2043406 auto_shard.cc:558] The `assert_cardinality` transformation is currently not handled by the auto-shard rewrite and will be removed.
2025-08-26 02:56:38.451593: I tensorflow/core/kernels/data/tf_record_dataset_op.cc:387] The default buffer size is 262144, which is overridden by the user specified `buffer_size` of 8388608


Epoch 1/12


2025-08-26 02:56:38.987791: W tensorflow/core/kernels/data/cache_dataset_ops.cc:916] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2025-08-26 02:56:38.989692: W tensorflow/core/kernels/data/cache_dataset_ops.cc:916] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
I0000 00:00:1756173400.215270 2043575 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1756173400.215278 2043573 cuda_dnn.cc:529] Loaded cuDNN ve

UnknownError: Graph execution error:

Detected at node adam/CollectiveReduceV2 defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/data/PythonEnvsSSD/tf_gpu_venv/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/data/PythonEnvsSSD/tf_gpu_venv/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/data/PythonEnvsSSD/tf_gpu_venv/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/data/PythonEnvsSSD/tf_gpu_venv/lib/python3.12/site-packages/tornado/platform/asyncio.py", line 211, in start

  File "/usr/lib/python3.12/asyncio/base_events.py", line 641, in run_forever

  File "/usr/lib/python3.12/asyncio/base_events.py", line 1987, in _run_once

  File "/usr/lib/python3.12/asyncio/events.py", line 88, in _run

  File "/data/PythonEnvsSSD/tf_gpu_venv/lib/python3.12/site-packages/ipyk